In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import cross_val_score
import joblib  # Model kaydetmek için

In [5]:
import pandas as pd

# CSV dosyasını yükleme
df_loaded = pd.read_csv('encoding_veriler.csv')

In [6]:
# Yeni özellikler oluşturalım
def create_special_features(df):
    # Burs öncelik puanı oluşturalım (başlangıçta herkes 0 puan)
    df['Burs_Oncelik_Puani'] = 0
    
    # 2. Kural: Özel durumu olanlar öncelikli (Özel Durumu Yok == 0, diğerleri == 1)
    df['Ozel_Durum_Oncelik'] = df['Özel_Durumu_Kodlanmış']  # Kodlanmış 0 ve 1 değerlerini alacak
    
    # 3. Kural: Ailevi durum etkisi
    df['Ailevi_Durum_Oncelik'] = (df['Ailevi durum ?'] != 'Anne ve baba birlikte').astype(int)
    
    # 4. Kural: Uzaktan/Açık öğretim engeli
    df['Egitim_Engeli'] = df['Öğretim durumu ?'].isin(['Açık öğretim', 'Uzaktan eğitim']).astype(int)
    
    # Burs öncelik puanını hesapla
    df['Burs_Oncelik_Puani'] += df['Ozel_Durum_Oncelik'] * 2  # Özel duruma 2 puan
    df['Burs_Oncelik_Puani'] += df['Ailevi_Durum_Oncelik']   # Ailevi duruma 1 puan
    
    # Burs engellerini kontrol et
    df['Burs_Engeli_Var'] = ((df['Burs_Engeli_Ayni_Sehir'] == 1) | 
                            (df['Egitim_Engeli'] == 1)).astype(int)
    
    return df

In [18]:
df_loaded.head(40)

,Eğitim durumunuz nedir?,Öğretim durumu ?,Yaşadığın Şehir?,Okuduğun şehir ?,Özel Durumu ?,Kaç kardeşiniz var? (Yoksa 0 girin.),Burs alan kardeşiniz oldu mu ?,Ailevi durum ?,Kira Durumunuz,Eviniz Var mı? kaç tane,...,Anne Mesleği,Baba Mesleği,Şehir Durumu,Özel_Durumu_Kodlanmış,Burs_Oncelik_Puani,Burs_Engeli_Ayni_Sehir,Ozel_Durum_Oncelik,Ailevi_Durum_Oncelik,Egitim_Engeli,Burs_Engeli_Var
0,0,0,1,10,0,1,0,3,0,0,...,1,0,0,1,3,0,1,1,0,0
1,0,0,2,10,1,1,0,1,1,1,...,2,7,0,0,1,0,0,1,0,0
2,0,0,3,10,1,2,0,1,0,1,...,2,3,0,0,1,0,0,1,0,0
3,0,0,2,12,1,2,1,1,1,1,...,3,3,0,0,1,0,0,1,0,0
4,0,0,1,1,1,1,0,1,1,2,...,4,7,1,0,1,1,0,1,0,1
5,0,0,4,10,1,4,0,1,1,1,...,2,3,0,0,1,0,0,1,0,0
6,0,0,5,29,1,1,0,0,1,1,...,2,1,0,0,1,0,0,1,0,0
7,0,0,6,7,1,3,0,1,1,1,...,3,7,0,0,1,0,0,1,0,0
8,0,0,7,7,1,1,1,1,1,0,...,2,3,1,0,1,1,0,1,0,1
9,0,0,8,7,1,1,0,1,0,1,...,2,8,0,0,1,0,0,1,0,0


In [7]:
# Yeni özellikleri ekle
df_enhanced = create_special_features(df_loaded)

# Modele verilecek veriyi hazırla
X = df_enhanced.drop(['Başvurduğunuz yıl size Devlet bursu mu yoksa Kredimi çıktı?'], axis=1)
y = df_enhanced['Başvurduğunuz yıl size Devlet bursu mu yoksa Kredimi çıktı?']

# Veriyi böl
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
print("Modelde kullanılan sütunlar:")
print(X.columns.tolist())


Modelde kullanılan sütunlar:
['Eğitim durumunuz nedir?', 'Öğretim durumu ?', 'Yaşadığın Şehir?', 'Okuduğun şehir ?', 'Özel Durumu ?', 'Kaç kardeşiniz var? (Yoksa 0 girin.)', 'Burs alan kardeşiniz oldu mu ?', 'Ailevi durum ?', 'Kira Durumunuz', 'Eviniz Var mı? kaç tane', 'Arabanız Var mı? kaç tane', 'Başvuru Yılı', '2020 Aylık Ev Geliri', '2021 Aylık Ev Geliri', '2022 Aylık Ev Geliri', '2023 Aylık Ev Geliri', '2024 Aylık Ev Geliri', 'Anne Mesleği', 'Baba Mesleği', 'Şehir Durumu', 'Özel_Durumu_Kodlanmış', 'Burs_Oncelik_Puani', 'Burs_Engeli_Ayni_Sehir', 'Ozel_Durum_Oncelik', 'Ailevi_Durum_Oncelik', 'Egitim_Engeli', 'Burs_Engeli_Var']


In [14]:
print("Modelde kullanılan sütunlar:")
print(X.columns.tolist())


Modelde kullanılan sütunlar:
['Eğitim durumunuz nedir?', 'Öğretim durumu ?', 'Özel Durumu ?', 'Kaç kardeşiniz var? (Yoksa 0 girin.)', 'Burs alan kardeşiniz oldu mu ?', 'Ailevi durum ?', 'Kira Durumunuz', 'Eviniz Var mı? kaç tane', 'Arabanız Var mı? kaç tane', 'Başvuru Yılı', '2020 Aylık Ev Geliri', '2021 Aylık Ev Geliri', '2022 Aylık Ev Geliri', '2023 Aylık Ev Geliri', '2024 Aylık Ev Geliri', 'Anne Mesleği', 'Baba Mesleği', 'Şehir Durumu', 'Özel_Durumu_Kodlanmış', 'Burs_Oncelik_Puani', 'Burs_Engeli_Ayni_Sehir', 'Ozel_Durum_Oncelik', 'Ailevi_Durum_Oncelik', 'Egitim_Engeli', 'Burs_Engeli_Var']


In [11]:
# Ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest modelini bu yeni özelliklerle eğit
rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42
)

In [12]:
# Modeli eğit
rf_model.fit(X_train_scaled, y_train)

# Tahminler
y_pred = rf_model.predict(X_test_scaled)

# Sonuçları göster
print("Model Performansı:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred))

Model Performansı:
Accuracy: 0.7272727272727273

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.60      0.43      0.50         7
           1       0.76      0.87      0.81        15

    accuracy                           0.73        22
   macro avg       0.68      0.65      0.66        22
weighted avg       0.71      0.73      0.71        22



In [13]:
# Özellik önemlerini göster
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nÖzellik Önem Sıralaması:")
print(feature_importances.head(15))

ValueError: All arrays must be of the same length